In [1]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. GPU:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available. Please check your installation.")


CUDA is available. GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")


e:\Bert\bert_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA is available. Using GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [3]:
# Load the IMDB dataset
dataset = load_dataset("imdb")

# Load the pre-trained tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Apply the tokenization function to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Check a few samples to see the tokenized data
print(tokenized_datasets['train'][0])


{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [4]:
# Define Training Arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="epoch",     # evaluate every epoch
    learning_rate=2e-5,              # learning rate for optimization
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    num_train_epochs=3,              # number of epochs to train
    weight_decay=0.01,               # strength of weight decay (regularization)
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# Initialize the model with pre-trained weights
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.to(device)  # Move model to the correct device (GPU or CPU)


e:\Bert\bert_env\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [5]:
# Initialize the Trainer
trainer = Trainer(
    model=model,                         # the model to train
    args=training_args,                  # training arguments
    train_dataset=tokenized_datasets['train'],  # training dataset
    eval_dataset=tokenized_datasets['test'],   # evaluation dataset
)

# Start the training process
trainer.train()

  0%|          | 10/9375 [00:20<5:16:30,  2.03s/it]

{'loss': 0.7036, 'grad_norm': 2.212522268295288, 'learning_rate': 1.997866666666667e-05, 'epoch': 0.0}


  0%|          | 20/9375 [00:40<5:19:30,  2.05s/it]

{'loss': 0.7041, 'grad_norm': 2.9303157329559326, 'learning_rate': 1.9957333333333336e-05, 'epoch': 0.01}


  0%|          | 30/9375 [01:00<5:15:50,  2.03s/it]

...

100%|█████████▉| 9370/9375 [6:14:06<00:10,  2.16s/it]

{'loss': 0.0007, 'grad_norm': 0.1327783763408661, 'learning_rate': 1.0666666666666668e-08, 'epoch': 3.0}


                                                     
100%|██████████| 9375/9375 [6:27:12<00:00,  2.48s/it]

{'eval_loss': 0.314434289932251, 'eval_runtime': 774.5093, 'eval_samples_per_second': 32.279, 'eval_steps_per_second': 4.035, 'epoch': 3.0}
{'train_runtime': 23232.5622, 'train_samples_per_second': 3.228, 'train_steps_per_second': 0.404, 'train_loss': 0.1730238575941324, 'epoch': 3.0}


TrainOutput(global_step=9375, training_loss=0.1730238575941324, metrics={'train_runtime': 23232.5622, 'train_samples_per_second': 3.228, 'train_steps_per_second': 0.404, 'total_flos': 1.9733329152e+16, 'train_loss': 0.1730238575941324, 'epoch': 3.0})

In [6]:
# Save the trained model and tokenizer
model.save_pretrained('./bert_imdb_model')
tokenizer.save_pretrained('./bert_imdb_model')


('./bert_imdb_model\\tokenizer_config.json',
 './bert_imdb_model\\special_tokens_map.json',
 './bert_imdb_model\\vocab.txt',
 './bert_imdb_model\\added_tokens.json')

In [7]:
# Evaluate the model on the test set
eval_results = trainer.evaluate()

# Print evaluation results
print("Evaluation results:", eval_results)


100%|██████████| 3125/3125 [12:53<00:00,  4.04it/s]

Evaluation results: {'eval_loss': 0.314434289932251, 'eval_runtime': 773.8351, 'eval_samples_per_second': 32.307, 'eval_steps_per_second': 4.038, 'epoch': 3.0}


In [8]:
# Function to make predictions
def predict(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():  # Disable gradient calculations for inference
        outputs = model(**inputs)
        predictions = outputs.logits.argmax(axis=-1)
    return predictions

# Example prediction
texts = ["I love this movie!", "This movie was terrible."]
predictions = predict(texts)

print("Predictions:", predictions)


Predictions: tensor([1, 0], device='cuda:0')


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class TextRequest(BaseModel):
    text: str

@app.post("/predict/")
def predict_api(request: TextRequest):
    inputs = tokenizer(request.text, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = outputs.logits.argmax(axis=-1).item()
    return {"prediction": prediction}

# Run the FastAPI app using the command below in your terminal (ensure the script is saved as `app.py`):
# uvicorn app:app --reload
